Import the pre trained model

In [1]:
import tensorflow as tf
import tensorflow.keras

In [2]:
model = tf.keras.models.load_model('trained_models/firstCNN.h5')
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 256, 256, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 128, 128, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 4096)              0

Import the pre processed data

In [6]:
import numpy as np
x_train = np.load('processed_data/first_experiment/x_im_train.npy')
x_test = np.load('processed_data/first_experiment/x_im_test.npy')
y_train = np.load('processed_data/first_experiment/y_im_train.npy')
y_test = np.load('processed_data/first_experiment/y_im_test.npy')

In [8]:
_, accuracy_test_original = model.evaluate(x_test, y_test)
print('Accuracy on original test data: {:4.2f}%'.format(accuracy_test_original * 100))

4/4 [==============================] - 6s 127ms/step - loss: 0.8203 - accuracy: 0.7345
Accuracy on original test data: 73.45%


Make ART model

In [9]:
from art.estimators.classification import TensorFlowV2Classifier
from art.attacks.evasion import FastGradientMethod, CarliniL0Method

C:\Users\Nicolas Facchinetti\.conda\envs\thesis\lib\site-packages\art\estimators\certification\__init__.py:13: UserWarning: PyTorch not found. Not importing DeepZ functionality
  warnings.warn("PyTorch not found. Not importing DeepZ functionality")


In [10]:
loss_object = tf.keras.losses.CategoricalCrossentropy()
classifier = TensorFlowV2Classifier(model=model, nb_classes=7, input_shape=(256, 256, 3), loss_object=loss_object)

In [11]:
attack_fgsm = FastGradientMethod(estimator=classifier, eps=0.3)

In [12]:
x_test_adv = attack_fgsm.generate(x_test)

In [13]:
_, accuracy_test_attack = model.evaluate(x_test_adv, y_test)
perturbation = np.mean(np.abs((x_test_adv - x_test)))

print('Accuracy on adversarial test data: {:4.2f}%'.format(accuracy_test_attack * 100))
print('Average perturbation: {:4.2f}'.format(perturbation))

4/4 [==============================] - 0s 53ms/step - loss: 23.7582 - accuracy: 0.0708
Accuracy on adversarial test data: 7.08%
Average perturbation: 0.27


In [15]:
attack_cw = CarliniL0Method(classifier=classifier)

x_test_adv = attack_cw.generate(x_test)

ResourceExhaustedError: OOM when allocating tensor with shape[113,256,256,16] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:Conv2D]

In [ ]:
loss_test, accuracy_test = model.evaluate(x_test_adv, y_test)
perturbation = np.mean(np.abs((x_test_adv - x_test)))
print('Accuracy on original test data: {:4.2f}%'.format(accuracy_test_original * 100))
print('Accuracy on adversarial test data: {:4.2f}%'.format(accuracy_test * 100))
print('Average perturbation: {:4.2f}'.format(perturbation))